In [13]:
import yaml

from redis import Redis
from lib.common import make_url, replace_env

In [14]:
CONFIG = '/config/todos/default_config.yml'
with open(CONFIG) as f:
    config = yaml.load(f,yaml.SafeLoader)
replace_env(config)

{'database': {'mongo': {'type': 'mongodb',
   'host': 'mongo',
   'user': 'mongouser',
   'password': 'averysecurepassword',
   'port': '27017',
   'db': 'todos_db'},
  'redis': {'type': 'redis',
   'host': 'redis',
   'port': '6379',
   'user': '',
   'password': 'redispassword',
   'db': '1'}}}

In [15]:
redis_config = config['database']['redis']

In [16]:
r = Redis(host=redis_config["host"],
    port=redis_config["port"],
    db=redis_config["db"],
    password=redis_config["password"])

In [17]:
r.keys()

[]

In [22]:
r.set('mkay','Helloooo', ex=10)

True

In [27]:
r.get('mkay')

In [29]:
d = {
    'a thing': 'here',
    'a number': 10
}

In [32]:
r.hmset('dic1',d)

True

In [35]:
r.hgetall('dic1')

{b'a thing': b'here', b'a number': b'10'}

In [53]:
d = {
    'uuid-1':{
        'val1':'value 1',
        'val2':'value 2'
    },'uuid-2':{
        'val1':'value 1',
        'val2':'value 2'
    },
}

In [55]:
json.loads(json.dumps(d))

{'uuid-1': {'val1': 'value 1', 'val2': 'value 2'},
 'uuid-2': {'val1': 'value 1', 'val2': 'value 2'}}

In [68]:
def redis_cachable(name,timeout=30):
    def _set_name(f):
        def _redis_cachable(self,uuid):
            key_name = name + '-' + uuid
            if r.exists(key_name):
                s = r.get(key_name)
                return json.loads(s)
            result = f(self,uuid)
            print(result)
            s = json.dumps(result)
            r.set(key_name,s,ex=timeout)
            return result
        return _redis_cachable
    return _set_name
        

In [93]:
import time
import json

class Resource:
    @redis_cachable('RESOURCE-GET',2)
    def get(self,uuid):
        return {'time': str(time.time())}
    

In [80]:
res = Resource()

In [92]:
res.get('some-id')

called
{'time': '1578499577.8837018'}


{'time': '1578499577.8837018'}